In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

#change path
root_dir = "/content/drive/My Drive/"
base_dir = root_dir + 'python_lesson/CloneCoding/펭귄 몸무게 예측 경진대회'
os.chdir(base_dir)

#mount google driver
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#root_dir = "/content/gdrive/My Drive/"

!pwd

/content/drive/My Drive/python_lesson/CloneCoding/펭귄 몸무게 예측 경진대회


# 펭귄 몸무게 예측 경진대회 - 기본 데이터 분석 & 예측

---

In [59]:
import pandas as pd

# csv 형식으로 된 데이터 파일을 읽어옵니다.
data = pd.read_csv('data/penguins.csv')

# 데이터의 최상단 5 줄을 표시합니다.
data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181,3750,Male
1,Adelie,Torgersen,39.5,17.4,186,3800,Female
2,Adelie,Torgersen,40.3,18.0,195,3250,Female
3,Adelie,Torgersen,36.7,19.3,193,3450,Female
4,Adelie,Torgersen,39.3,20.6,190,3650,Male


### 결측치 확인

In [60]:
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(data)

결측치가 존재하지 않습니다


### 결측치 처리

결측치 존재하지 않으므로 패스

In [61]:
# 결측치가 있는 row들을 확인합니다.
train[data.isna().sum(axis=1) > 0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex


In [62]:
# 결측치를 처리하는 함수를 작성합니다.
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O':
            # 카테고리형 feature가 결측치인 경우 해당 행들을 삭제해 주었습니다.
            temp = temp.dropna(subset=[col])
        elif dtype == int or dtype == float:
            # 수치형 feature가 결측치인 경우 평균값을 채워주었습니다.
            temp.loc[:,col] = temp[col].fillna(temp[col].mean())
    return temp

data = handle_na(data, missing_col)

# 결측치 처리가 잘 되었는지 확인해 줍니다.
missing_col = check_missing_col(data) 

결측치가 존재하지 않습니다


### 데이터 분할

In [63]:
# train/test로 분할

from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, shuffle=True)

print(train.shape)
print(test.shape)

(266, 7)
(67, 7)


### 데이터 설명 방정식 만들기(간단하게)

$$y = a_1x_1+a_2x_2+a_3x_3 + b$$

In [64]:
train.columns

Index(['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex'],
      dtype='object')

In [66]:
# X와 Y를 각각 따로 저장합니다.
target = ['bill_length_mm','bill_depth_mm','flipper_length_mm']
X = train[target]
Y = train['body_mass_g']

X

,bill_length_mm,bill_depth_mm,flipper_length_mm
309,50.9,17.9,196
99,39.7,18.9,184
184,48.7,15.7,208
169,46.3,15.8,215
161,49.3,15.7,217
...,...,...,...
246,47.2,15.5,215
180,48.4,16.3,220
24,40.5,18.9,180
201,43.8,13.9,208


#### a 구하기

먼저 부리의 길이, 부리의 깊이, 날개의 길이와 몸무게가 어떤 관계를 갖는지, 그 평균 값을 구해줍니다.

---
❗셔플링을 통해 분할하기 때문에 약간의 값 차이가 있을 수 있습니다.

In [72]:
import numpy as np

X_mean = np.mean(X)
Y_mean = np.mean(Y)

print(f"부리 길이의 평균:{X_mean['bill_length_mm']:.2f}")
print(f"부리 깊이의 평균:{X_mean['bill_depth_mm']:.2f}")
print(f"날개 길이의 평균:{X_mean['flipper_length_mm']:.2f}")
print(f"몸무게의 평균:{Y_mean:.2f}")

print(f"따라서 몸무게는 부리 길이의 평균 {Y_mean/X_mean['bill_length_mm']:.2f}배 입니다.")
print(f"따라서 몸무게는 부리 깊이의 평균 {Y_mean/X_mean['bill_depth_mm']:.2f}배 입니다.")
print(f"따라서 몸무게는 날개 길이의 평균 {Y_mean/X_mean['flipper_length_mm']:.2f}배 입니다.")

부리 길이의 평균:44.24
부리 깊이의 평균:17.18
날개 길이의 평균:201.33
몸무게의 평균:4240.04
따라서 몸무게는 부리 길이의 평균 95.84배 입니다.
따라서 몸무게는 부리 깊이의 평균 246.79배 입니다.
따라서 몸무게는 날개 길이의 평균 21.06배 입니다.


이렇게 보면 $y$는 $x_1$의 96배 정도의 크기를 가지고 있다고 생각할 수 있습니다. <br>
마찬가지로 $y$는 $x_2$의 247배, $x_3$의 21배 정도의 크기를 가지고 있다고 생각할 수 있습니다. <br>

$$y = 95.84x_1+246.79x_2+21.06x_3+b$$


### b 구하기

$a$는 $X$와 $Y$의 강력한 관계를 나타냈다면, $b$는 그보다 약한 관계를 나타냅니다. <br>
$a$ 만으로는 설명하지 못하는 부분을 구하는 과정이라고 볼 수 있다. <br>
$a$를 넣어서 새롭게 업데이트 된 방정식을 사용하면 $b$를 구할 수 있습니다.

$$y-95.84x_1-246.79x_2-21.06x_3=b$$

In [73]:
b = Y_mean-95.84*X_mean['bill_length_mm']-246.79*X_mean['bill_depth_mm']-21.06*X_mean['flipper_length_mm']

print(f"업데이트 된 방정식의 b는 {b:.2f}입니다.")

업데이트 된 방정식의 b는 -8480.02입니다.


### $Y=95.84x_1+246.79x_2+21.06x_3-8480.02$

이 식을 통해 결과 값들을 예측해봅시다.

In [74]:
predict_Y = 95.84*X['bill_length_mm']+246.79*X['bill_depth_mm']+21.06*X['flipper_length_mm'] - 8480.02

predict_Y

309    4943.537
99     3864.199
184    4442.471
169    4384.554
161    4689.515
         ...   
246    4396.773
180    4814.513
24     3856.631
201    3528.633
2      3931.252
Length: 266, dtype: float64

### 예측 결과 평가하기

대회 평가 지표(metric)는 RMSE, Root Mean of Squared Error 입니다.

In [75]:
# 대회 규칙의 평가 산식 함수를 그대로 사용합니다.

def RMSE(true, pred):
  score =np.sqrt(np.mean(np.square(true-pred)))
  return score

In [87]:
# 실제 정답 값
real_answer = Y.copy()

# 정답과 예측 값을 함수에 넣어 결과를 확인합니다.
error = RMSE(real_answer, predict_Y)

print(f"이 방정식의 평균 에러는 {error:.2f} 입니다.")

이 방정식의 평균 에러는 800.33 입니다.


### test_data 예측하기

test 데이터에 Y값이 없다고 가정하고 실제로 주어지지 않은 데이터를 예측해보겠습니다.

In [79]:
test

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
270,Chinstrap,Dream,45.2,17.8,198,3950,Female
97,Adelie,Biscoe,37.8,20.0,190,4250,Male
205,Gentoo,Biscoe,45.3,13.8,208,4200,Female
223,Gentoo,Biscoe,45.2,16.4,223,5950,Male
109,Adelie,Biscoe,42.7,18.3,196,4075,Male
...,...,...,...,...,...,...,...
261,Gentoo,Biscoe,46.8,14.3,215,4850,Female
124,Adelie,Torgersen,38.5,17.9,190,3325,Female
48,Adelie,Biscoe,34.5,18.1,187,2900,Female
216,Gentoo,Biscoe,48.2,15.6,221,5100,Male


In [81]:
test_X = test[target]

predict_test = 95.84*test_X['bill_length_mm']+246.79*test_X['bill_depth_mm']+21.06*test_X['flipper_length_mm'] - 8480.02

predict_test

270    4414.690
97     4079.932
205    3647.714
223    4595.684
109    4256.365
         ...   
261    4062.289
124    3628.761
48     3231.579
216    4643.652
175    4504.419
Length: 67, dtype: float64

### (한 걸음 더) 방정식으로 모델 학습 맛보기

위 식보다 적은 에러를 내는, 더 정확한 식을 찾아보자. <br>
그러기 위해선 이 방정식의 a와 b를 변화시키면서 더 정확한 식을 찾아야한다.

$$𝑌=95.84𝑥1+246.79𝑥2+21.06𝑥3−8480.02$$

In [82]:
# 탐색할 a와 b의 후보들을 마련합니다.
a1_list = [95.54, 95.64, 95.74, 95.84, 95.94, 96.04] # 0.1 단위로 변화
a2_list = [246.49, 246.59, 246.69, 246.79, 246.89, 246.99] # 0.1 단위로 변화
a3_list = [20.76, 20.86, 20.96, 21.06, 21.16, 21.26] # 0.1 단위로 변화
b_list = [8479.82, 8479.92, 8479.02, 8479.12, 8479.22, 8479.32] # 0.1 단위로 변화

In [89]:
# 각각의 후보자들 중에서 가장 에러가 적은 후보를 고르는 함수를 작성합니다.
def find_best_params(a1_list, a2_list, a3_list, b_list, X, Y):
    min_error = 99999999

    for a1 in a1_list:
      for a2 in a2_list:
        for a3 in a3_list:
          for b in b_list:
            predict = a1 * X['bill_length_mm'] + a2 * X['bill_depth_mm'] + a3 * X['flipper_length_mm'] - b
            error = RMSE(Y, predict)
            
            if error < min_error:
                print("모델 개선이 가능합니다.")
                print(f"a1={a1}, a2={a2}, a3={a3}, b={b}")
                print(f"error={error:.5f}")
                min_error = error
                best_a1, best_a2, best_a3, best_b = a1, a2, a3, b
                
    print(f"최적의 parameter는 a1 = {best_a1}, a2 = {best_a2}, a3 = {best_a3}, b = {best_b}\n이때 평균 에러는 {min_error:.2f} 입니다.")
    return best_a1, best_a2, best_a3, best_b, min_error

a1, a2, a3, b, min_error = find_best_params(a1_list, a2_list, a3_list, b_list, X, Y)
print(f"기존 방정식의 평균 에러 : {error:.2f}")
print(f"모델 성능 개선 : {error - min_error:.2f}")

모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=20.76, b=8479.82
error=805.32792
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=20.76, b=8479.02
error=805.25021
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=20.86, b=8479.82
error=803.00107
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=20.86, b=8479.02
error=802.94319
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=20.96, b=8479.82
error=801.17608
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=20.96, b=8479.02
error=801.13818
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=21.06, b=8479.82
error=799.85640
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=21.06, b=8479.02
error=799.83857
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=21.16, b=8479.82
error=799.04452
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=21.16, b=8479.92
error=799.04428
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=21.26, b=8479.82
error=798.74199
모델 개선이 가능합니다.
a1=95.54, a2=246.49, a3=21.26, b=8479.92
error=798.73924
최적의 parameter는 a1 = 95.54, a2 = 246.49, a3 = 21.26, b = 8479.92
이때 평균 에러는 798.74 입니다.
기존 방정식의 평균 에러 : 800.33
모델 성능 개선 : 1.59


## Reference

1. https://www.dacon.io/competitions/official/235862/codeshare/4031?page=1&dtype=recent
2. https://www.kaggle.com/resulcaliskan/penguins